1. **Симуляция секвенирования** (3 балла)

Первое задание заключается в том, чтобы написать симуляцию секвенирования ридов при помощи illumina. Для этого необходимо сгенерировать случайно строку длинной 50 000 символов, над алфавитом {A, T, G, C}. После этого случайно выбирать подстроки, длинна которых распределена нормально со средним 250 и среднеквадратическим отклонением 30, каждую такую подстроку "секвенировать".

Под словом "секвенировать" в данном случае имеется ввиду симуляция того процесса, который мы обсуждали на лекции. Вы считываете очередной символ подстроки, например, 100 раз, но в N случаях из 100 ошибаетесь и считываете любой нуклеотид, кроме правильного с одинаковой вероятностью. N принадлежит равномерному распределению от 0 до 60. По получившемуся набору вычисляете наибоее вероятный нуклеотид (тот, которого больше всего) и его качество прочтения, чтобы записать его в формате FASTQ.
При симуляции важно запоминать ID рида и для каждого ID позиции, в которых нуклеотид был считан неверно и какой должен быть на самом деле (для этого просто можно хранить 2 числа и помнить что у вас есть исходная строка, откуда берутся риды). Это понадобится для выполнения следующих заданий.
Всего ридов пусть будет 50К.

2. **Удаление ошибок (Trimmomatic)** (2 балла) 

Получившиеся риды обработайте Trimmomatic со стандартными параметрами с лекции. Пользуясь тем, что вы знаете, для каждого рида ground truth, посчитайте, в скольких случаях Trimmomatic удалил нуклеотиды, которые были считаны верно, а в скольких действительно удалил ошибочные.

3. **Коррекция ошибок** (4 балла) 

Получившиеся риды обработайте одним(любым) из инструментов, которые использованы и проанализированны в этой статье. Посчитайте TP, FP, TN, FN.

In [ ]:
import math
import random
import subprocess
from collections import Counter


ALPHABET = ['A', 'C', 'G', 'T']


if __name__ == '__main__':
    sequence = "".join([random.choice(ALPHABET) for _ in range(10000)])
    print("Sequence is generated")

    correct_reads = []
    reads = []
    quality = []
    for read_id in range(5000):
        l = int(random.gauss(250, 30))
        pos = random.randint(0, len(sequence) - l)
        r = []
        q = []
        for i in range(l):
            errors = random.randint(0, 70)
            variants = [sequence[pos + i]] * (100 - errors)
            while len(variants) < 100:
                wrong_letter = random.choice(ALPHABET)
                if wrong_letter == sequence[pos + i]:
                    continue
                variants.append(wrong_letter)
            res_letter, res_count = Counter(variants).most_common(1)[0]
            r.append(res_letter)
            q.append(chr(int(-10 * math.log10(max(0.00001, 1 - res_count / 100))) + 33))
        correct_reads.append(sequence[pos:pos + l])
        reads.append("".join(r))
        quality.append("".join(q))

    with open('input.fastq', 'w') as input_file:
        for i in range(len(reads)):
            input_file.write("@" + str(i) + "\n")
            input_file.write(reads[i] + "\n")
            input_file.write("+\n")
            input_file.write(quality[i] + "\n")

    def count_correct_reads(filename):
        with open(filename, 'r') as _file:
            trimmed_data = list(map(str.strip, _file.readlines()))
        matrix = [[0, 0], [0, 0]]
        for i in range(0, len(trimmed_data), 4):
            id = int(trimmed_data[i][1:])
            read = trimmed_data[i + 1]
            correct_before = 1 if reads[id] == correct_reads[id] else 0
            correct_after = 1 if read == correct_reads[id] else 0
            matrix[correct_before][correct_after] += 1
        print(f"File {filename}:")
        for i in range(2):
            for j in range(2):
                print("Correctness before =", i, "Correctness after =", j, "Count:", matrix[i][j])
        print("Overall", "Incorrect:", matrix[0][0] + matrix[1][0], "Correct:", matrix[0][1] + matrix[1][1])

    print("Original data")
    count_correct_reads("input.fastq")
    print()

    subprocess.run(
        "java -jar trimmomatic-0.39.jar SE -phred33 input.fastq trimmed.fastq "
        "LEADING:1 TRAILING:1 SLIDINGWINDOW:5:2 MINLEN:200".split(" ")
    )
    print("Trimmomatic is used")
    count_correct_reads("trimmed.fastq")
    print()

    subprocess.run(
        "./pollux -i trimmed.fastq -o p_out -k 25 -n false -d false -h false".split(" ")
    )
    print("Pollux is used")
    count_correct_reads("p_out/trimmed.fastq.corrected")
    print()

In [ ]:
# Sequence is generated
# Original data
# File input.fastq:
# Correctness before = 0 Correctness after = 0 Count: 2462
# Correctness before = 0 Correctness after = 1 Count: 0
# Correctness before = 1 Correctness after = 0 Count: 0
# Correctness before = 1 Correctness after = 1 Count: 2538
# Overall Incorrect: 2462 Correct: 2538
#
# TrimmomaticSE: Started with arguments:
#  -phred33 input.fastq trimmed.fastq LEADING:1 TRAILING:1 SLIDINGWINDOW:5:2 MINLEN:200
# Automatically using 4 threads
# Input Reads: 5000 Surviving: 2554 (51,08%) Dropped: 2446 (48,92%)
# TrimmomaticSE: Completed successfully
# Trimmomatic is used
# File trimmed.fastq:
# Correctness before = 0 Correctness after = 0 Count: 1221
# Correctness before = 0 Correctness after = 1 Count: 0
# Correctness before = 1 Correctness after = 0 Count: 294
# Correctness before = 1 Correctness after = 1 Count: 1039
# Overall Incorrect: 1515 Correct: 1039
#
#
# Pollux 1.0.2
# Source compiled on Apr 19 2023 at 22:49:09.
#
# Considering argument: -i : number of input files is 1
# Considering argument: -o : output directory is p_out
# Considering argument: -k : k-mer size is 25
# Considering argument: -n : insertions disabled
# Considering argument: -d : deletions disabled
# Considering argument: -h : homopolymers disabled
#
# File name: trimmed.fastq
#
# ERROR CORRECTION
#
# Creating read objects...
# Reading file: trimmed.fastq
# Finished creating read objects!
#
# Constructing k-mers...
# Processing file 1/1...
# 0% 5% 10% 15% 20% 25% 30% 35% 40% 45% 50% 55% 60% 65% 70% 75% 80% 85% 90% 95% 100%
# Preprocessing k-mers...
# 0% 5% 10% 15% 20% 25% 30% 35% 40% 45% 50% 55% 60% 65% 70% 75% 80% 85% 90% 95% 100%
# Removed 65688 unique k-mers from the set of 87214 total k-mers.
# Resizing...
# Finished resizing...
# Low k-mer count value was observed to be 5.
# Finished preprocessing k-mers!
#
# Finished constructing k-mers!
#
# Correcting reads...
# Correcting file 1/1...
# 0% 5% 10% 15% 20% 25% 30% 35% 40% 45% 50% 55% 60% 65% 70% 75% 80% 85% 90% 95% 100%
#
# Corrected...
# Reads with Multiple Corrections: 300
#
# Substitution Corrections: 1546
# Single Insertion Corrections: 0
# Single Deletion Corrections: 0
#
# Total Homopolymer Corrections: 0
#
# Breakdown:
# < -10: 0
# -10 : 0
# -9 : 0
# -8 : 0
# -7 : 0
# -6 : 0
# -5 : 0
# -4 : 0
# -3 : 0
# -2 : 0
# -1 : 0
# 1 : 0
# 2 : 0
# 3 : 0
# 4 : 0
# 5 : 0
# 6 : 0
# 7 : 0
# 8 : 0
# 9 : 0
# 10 : 0
# > 10: 0
#
# Total Insertions (Single + Homopolymer): 0
# Total Deletions (Single + Homopolymer): 0
#
# Finished correcting reads!
#
# Pollux is used
# File p_out/trimmed.fastq.corrected:
# Correctness before = 0 Correctness after = 0 Count: 320
# Correctness before = 0 Correctness after = 1 Count: 901
# Correctness before = 1 Correctness after = 0 Count: 294
# Correctness before = 1 Correctness after = 1 Count: 1039
# Overall Incorrect: 614 Correct: 1940

**Симуляция секвенирования**

Так как 50к ридов слишком долго считалось, их количество было уменьшено до 5к, без существенной потери в информативности.

**Удаление ошибок**

Стандартные параметры с лекции были ослаблены, чтобы получить какую-то адекватную информацию о неправильных прочтениях. Результат неудовлетворительный, так как слишком часто удаляет риды, которые считает плохими (выкинул половину всех ридов), что происходит из-за большой веротяности ошибки (до 60%). Кроме того он ничего не исправил (итоговое количество incorrect = 1515), при этом испортил 294 изначально правильных.

**Коррекция ошибок**

Для обработки выбрала Pollux: https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-014-0435-6

ссылка на гит: https://github.com/emarinier/pollux .

Результат получился лучше, чем с использованием Trimmomatic. В примере исправил 901 изначально неправильных, при этом испортил только 294. 